## 23.3 Knapsack

The 0/1 knapsack problem asks for the most valuable subset of items that
doesn't exceed a given weight limit. The problem can be solved with
[exhaustive search](../11_Search/11_5_subsets.ipynb#Exercise-11.5.2) or
[backtracking](../22_Backtracking/22_7_knapsack.ipynb#22.7-Back-to-the-knapsack) but not with
[greed](../18_Greed/18_1_scheduling.ipynb#Exercise-18.1.1).
Let's now consider dynamic programming.

Bottom-up dynamic programming is usually implemented with for-loops
iterating over an index-based cache.
Sets can't be indexed, but sequences can.
So let's slightly modify the formulation of the problem,
using sequences and subsequences instead of sets and subsets.

**Function**: knapsack\
**Inputs**: *items*, a sequence of pairs of integers; *capacity*, an integer\
**Preconditions**:
for each (*weight*, *value*) in *items*, *weight* > 0 and *value* > 0;
*capacity* ≥ 0\
**Output**: *packed*, a sequence of pairs of integers\
**Postconditions**:

- *packed* is a subsequence of *items*
- the sum of the weights in *packed* doesn’t exceed *capacity*
- the sum of the values in *packed* is largest among all sequences satisfying the previous two conditions

Here are the necessary constants and functions and some tests.

In [1]:
from algoesup import check_tests, test

WEIGHT = 0
VALUE = 1


def weight(items: list) -> int:
    """Return the total weight of the items."""
    total = 0
    for item in items:
        total = total + item[WEIGHT]
    return total


def value(items: list) -> int:
    """Return the total value of the items."""
    total = 0
    for item in items:
        total = total + item[VALUE]
    return total


ITEMS = [(2, 3), (2, 4), (3, 4), (4, 20), (5, 30)]
knapsack_tests = [
    # case,         items,  capacity,   knapsack
    ('none fits',   ITEMS,  1,          []),
    ('all fit',     ITEMS,  16,         ITEMS),
    ('one is best', ITEMS,  6,          [(5, 30)])
]

check_tests(knapsack_tests, [list, int, list])

OK: the test table passed the automatic checks.


### 23.3.1 Recursive

We must first recursively define the function we want to compute,
namely knapsack(*items*, *capacity*).

As usual, we start with the bases cases.
The two smallest possible inputs correspond to no items and no capacity.
In both cases nothing can be put in the knapsack:
the output is the empty sequence.

- if *items* = (): knapsack(*items*, *capacity*) = ()
- if *capacity* = 0: knapsack(*items*, *capacity*) = ()

If *items* isn't the empty sequence, we can separate it into
a head (the next item to consider) and a tail (the other items).
Like for the backtracking solution, there are only two cases:
either the head item fits in the knapsack or it doesn't. If the latter, i.e.
the item weighs more than the remaining capacity, it must be skipped.

- if weight(head(*items*)) > *capacity*:
  knapsack(*items*, *capacity*) = knapsack(tail(*items*), *capacity*)

If the item fits, we don't know whether we should put it in the knapsack or not,
so we try both options and choose the one that leads to
the most valuable knapsack.
To make the final recurrence relation less verbose and easier to read,
I use *item* as an abbreviation of head(*items*).

- otherwise: knapsack(*items*, *capacity*) = most valuable of
  - knapsack(tail(*items*), *capacity*) and
  - (*item*) + knapsack(tail(*items*), *capacity* − weight(*item*))

The + operator is again the concatenation operator.
If the item is put in the knapsack, the remaining capacity is decreased.
Here's my implementation.

In [2]:
def knapsack(items: list, capacity: int) -> list:
    """Return a highest-value subsequence of items that weigh at most capacity.

    Preconditions:
    - items is a list of weight-value pairs, both positive integers
    - capacity ≥ 0
    """
    if len(items) == 0 or capacity == 0:
        return []
    else:
        item = items[0]  # head
        rest = items[1:]  # tail
        skip = knapsack(rest, capacity)
        # if item doesn't fit, we must skip it
        if item[WEIGHT] > capacity:
            return skip
        # otherwise take it if that leads to a higher value
        else:
            take = [item] + knapsack(rest, capacity - item[WEIGHT])
            if value(skip) > value(take):
                return skip
            else:
                return take


test(knapsack, knapsack_tests)

Testing knapsack...
Tests finished: 3 passed (100%), 0 failed.


The next step, in preparation for dynamic programming, is to use indices.

#### Exercise 23.3.1

Copy the code above into the auxiliary function below and modify it so that
it uses the `index` argument instead of repeatedly slicing the `items` list.
Add the call to the auxiliary function.

In [3]:
def knapsack_indices(items: list, capacity: int) -> list:  # noqa: D103
    # docstring not repeated

    def knapsack(index: int, capacity: int) -> list:
        """Return a subsequence of items[index:].

        Preconditions: 0 ≤ index ≤ len(items) and 0 ≤ capacity
        Postconditions: the output fits the capacity and maximises the value
        """
        pass

    pass  # call the auxiliary function and return the solution


test(knapsack_indices, knapsack_tests)

[Hint](../31_Hints/Hints_23_3_01.ipynb)
[Answer](../32_Answers/Answers_23_3_01.ipynb)

### 23.3.2 Common subproblems

Like for the Fibonacci and LCS problems, the next step is to think whether there
are common subproblems that would benefit from caching the solutions.

Is it possible for the recursive function
`knapsack(items, capacity)` or `knapsack(index, capacity)` to be called
several times with the exact same input values?
(Hint: consider the list of items `[(1, 3), (1, 4), (3, 4), (4, 20)]`.)

___

In the LCS problem, skipping first the left head and then the right head
or vice versa leads to the same subproblem. Here we can have the same issue.
If two items have the same weight, like the two items with weight 1 in the hint,
then whether we skip the first and take the second or
take the first and skip the second, we will arrive at the same subproblem,
with the same remaining items and capacity.
For the example in the hint, the recursive algorithm twice solves the subproblem
`knapsack([(3, 4), (4, 20)], capacity - 1)`,
or `knapsack(2, capacity - 1)` in the version with indices.
This means that any further subproblem will also be solved at least twice.
Twice the item with weight 3 will be taken to solve
`knapsack([(4, 20)], capacity - 4)` and twice that item will be skipped to
solve `knapsack([(4, 20)], capacity - 1)`, and so on.
If we draw the DAG we can easily see how many paths lead to each subproblem.

#### Exercise 23.3.2

1. Draw the DAG of the subproblems of
   knapsack([(1, 3), (1, 4), (3, 4), (4, 20)], 4).

To make the DAG less tedious to write, I suggest you omit 'knapsack' and
just write a list–capacity pair or, even shorter, an index–capacity pair.
Here are three of the DAG's nodes:

Subproblem | Shorthand | Notes
-|-|-
([(1, 3), (1, 4), (3, 4), (4, 20)], 4) | (0, 4) | no item processed, capacity is 4
([(1, 4), (3, 4), (4, 20)], 4) | (1, 4) | first item skipped, capacity still 4
([(1, 4), (3, 4), (4, 20)], 3) | (1, 3) | first item added, reducing capacity by 1

2. After drawing the DAG, find which subproblems are solved more than once.

_Write your answer here._

[Hint](../31_Hints/Hints_23_3_02.ipynb)
[Answer](../32_Answers/Answers_23_3_02.ipynb)

### 23.3.3 Top-down and bottom-up

The next step is to think about the cache, which is the same for the
top-down and bottom-up approaches. As always, the cache stores the solution of
each problem instance. For this problem, the cache stores lists of items,
namely the most valuable knapsack for each problem instance, which is
a pair of integers: the current index and the remaining capacity.

What's the best way to implement the cache?
With a hash table (Python dictionary), an array or a matrix?
If it's a hash table, what are the keys? If it's an array or matrix,
what do the indices represent and how many are there,
i.e. what is the size of the array or matrix?

___

Like the LCS problem, this one has two inputs that are natural numbers and
can be used as indices to look up the solution in a matrix:
`cache[i][c]` is the solution for instance knapsack(*i*, *c*), i.e. it's
the most valuable subsequence of `items[i:]` that doesn't exceed weight `c`.

The row index goes from 0 to │*items*│ and is the index of the current item.
The column index goes from 0 to *capacity* and indicates the remaining capacity.

I could instead use columns for item indices and rows for capacities,
but I find `cache[i][c]` more intuitive than `cache[c][i]` because
in the former the matrix indices follow the same order as the problem inputs.

#### Exercise 23.3.3

Copy your code for Exercise 23.3.1 and modify it so that
it creates and uses a cache in a top-down fashion.

In [4]:
def knapsack_topdown(items: list, capacity: int) -> list:  # noqa: D103
    def knapsack(index: int, capacity: int) -> list:
        """Return a subsequence of items[index:].

        Preconditions: 0 ≤ index ≤ len(items) and 0 ≤ capacity
        Postconditions: the output fits the capacity and maximises the value
        """
        pass

    pass  # create an empty cache and call the auxiliary function

If you add a print statement to trace how the cache is filled, run

In [5]:
knapsack_topdown([(1, 3), (1, 4), (3, 4), (4, 20)], 4)

and check the caching follows a topological sort of the DAG for
[Exercise 23.3.2](../32_Answers/Answers_23_3_02.ipynb).

Afterwards, uncomment your print statement before running all tests.

In [6]:
test(knapsack_topdown, knapsack_tests)

[Hint](../31_Hints/Hints_23_3_03.ipynb)
[Answer](../32_Answers/Answers_23_3_03.ipynb)

The next step is to think in which order to fill the cache for the
bottom-up approach. Here again are the recurrence relations,
with the parts that don't relate to subproblem dependencies omitted:

- if ...:
  knapsack(*items*, *capacity*) = knapsack(tail(*items*), *capacity*)
- otherwise: knapsack(*items*, *capacity*) = ... of
  - knapsack(tail(*items*), *capacity*) and
  - ... + knapsack(tail(*items*), *capacity* − weight(*item*))

If we convert the above to matrix cells, which cells does `cache[i][c]`
possibly depend on? (Hint: reformulate the recurrence relations using indices.)

___

The recurrence relations (and the code) tell us that knapsack(*i*, *c*)
depends on knapsack(*i* + 1, *c*) and knapsack(*i* + 1, *c* − *w*),
where *w* is the weight of *items*[*i*].
This means that each matrix cell depends on cells in the next row and
in columns to the left (lower-capacity value).

Therefore, in which order should the rows of the matrix be filled?
And in which order should the columns be filled?

___

The rows must be filled from last to first. As for the columns,
they must be filled first to last.

#### Exercise 23.3.4

Copy your top-down dynamic programming approach to the next cell and modify it
so that it fills the cache iteratively.

In [7]:
def knapsack_bottomup(items: list, capacity: int) -> list:  # noqa: D103
    # create the cache
    # for each row from last to first:
        # for each column:
            # compute cache[row][column]
    pass # return the cell with the solution for items and capacity

test(knapsack_bottomup, knapsack_tests)

[Hint](../31_Hints/Hints_23_3_04.ipynb)
[Answer](../32_Answers/Answers_23_3_04.ipynb)

If you haven't attempted the exercises,
I suggest you copy my solutions to this notebook,
so that they are next to each other.
That way you will see that the recursive, top-down and bottom-up
algorithms are mostly the same, and that
each one can be systematically derived from the previous one.

### 23.3.4 Complexity

As always, the worst-case top-down and bottom-up complexities are the same:
the matrix size multiplied by the worst-case complexity of filling each cell.

What's the size of the matrix, in terms of │*items*│ and *capacity*?
What's the worst-case complexity of filling each cell?
(Hint: the reasoning is similar to the LCS problem.)

___

The matrix has (│*items*│ + 1) × (*capacity* + 1) cells.
In the worst case, a cell requires

- concatenating the head item with the best knapsack for the tail and remaining capacity
- comparing the values of two knapsacks (with and without the item).

A knapsack can never have more items than the input list, so the worst-case
complexity for computing the value of a cell is Θ(│*items*│).
The overall worst-case complexity is
(│*items*│ + 1) × (*capacity* + 1) × Θ(│*items*│) = Θ(│*items*│² × *capacity*).

This is usually much better than the exponential complexity for generating
and testing all subsequences of *items*. However, as the capacity approaches
2ⁿ, with *n* = │*items*│, the dynamic approach may not be much faster.
For example, if we have ten items to put in a container with 1000 kg capacity,
then the cache has over ten thousand cells. Filling them all in the bottom-up
approach may take longer than generating the $2^{10} = 1024$ subsets of
ten items.

⟵ [Previous section](23_2_lcs.ipynb) | [Up](23-introduction.ipynb) | [Next section](23_4_summary.ipynb) ⟶